# Physical Constraint Normalization + Stacking Meta-Learner

**Hypothesis**: 
1. Physical constraint enforcement (sum ≤ 1) could improve generalization
2. MLP meta-learner on OOF predictions could learn better weights

**Key Finding from Analysis**:
- 272 rows (14.4%) in current submission violate physical constraint (sum > 1)
- Max sum is 1.1323
- This is the ONLY untried approach that directly addresses generalization

**Implementation**:
1. Train GP, MLP, LGBM separately
2. Generate out-of-fold predictions
3. Train MLP meta-learner on OOF predictions
4. Apply physical constraint normalization to final predictions

In [1]:
# Standard imports
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import lightgbm as lgb
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel
from sklearn.preprocessing import StandardScaler
import tqdm
import warnings
warnings.filterwarnings('ignore')

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.set_default_dtype(torch.double)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [2]:
# Data loading functions
DATA_PATH = '/home/data'

INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_FULL_SOLVENT = ["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

print('Data loading functions defined')

Data loading functions defined


In [3]:
# Load feature lookups
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
DRFP_DF = pd.read_csv(f'{DATA_PATH}/drfps_catechol_lookup.csv', index_col=0)
ACS_PCA_DF = pd.read_csv(f'{DATA_PATH}/acs_pca_descriptors_lookup.csv', index_col=0)

# Filter DRFP to high-variance columns
drfp_variance = DRFP_DF.var()
nonzero_variance_cols = drfp_variance[drfp_variance > 0].index.tolist()
DRFP_FILTERED = DRFP_DF[nonzero_variance_cols]

print(f'Spange: {SPANGE_DF.shape}, DRFP filtered: {DRFP_FILTERED.shape}, ACS PCA: {ACS_PCA_DF.shape}')

Spange: (26, 13), DRFP filtered: (24, 122), ACS PCA: (24, 5)


In [4]:
# Full Featurizer - 145 features
class FullFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.drfp_df = DRFP_FILTERED
        self.acs_pca_df = ACS_PCA_DF
        self.feats_dim = 2 + 3 + self.spange_df.shape[1] + self.drfp_df.shape[1] + self.acs_pca_df.shape[1]

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            A_drfp = self.drfp_df.loc[X["SOLVENT A NAME"]].values
            B_drfp = self.drfp_df.loc[X["SOLVENT B NAME"]].values
            A_acs = self.acs_pca_df.loc[X["SOLVENT A NAME"]].values
            B_acs = self.acs_pca_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
                X_drfp = B_drfp * (1 - (1-pct)) + A_drfp * (1-pct)
                X_acs = B_acs * (1 - (1-pct)) + A_acs * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
                X_drfp = A_drfp * (1 - pct) + B_drfp * pct
                X_acs = A_acs * (1 - pct) + B_acs * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
            X_drfp = self.drfp_df.loc[X["SOLVENT NAME"]].values
            X_acs = self.acs_pca_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange, X_drfp, X_acs])

# Simple Featurizer (for GP) - 18 features
class SimpleFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.feats_dim = 2 + 3 + self.spange_df.shape[1]

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange])

print(f'Full feature dimension: {FullFeaturizer().feats_dim}')
print(f'Simple feature dimension: {SimpleFeaturizer().feats_dim}')

Full feature dimension: 145
Simple feature dimension: 18


In [5]:
# MLP Model
class MLPModel(nn.Module):
    def __init__(self, input_dim=145, hidden_dim=64, output_dim=3, dropout=0.2):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim // 2)
        self.fc3 = nn.Linear(hidden_dim // 2, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = torch.sigmoid(self.fc3(x))
        return x

# Meta-Learner MLP (smaller, for stacking)
class MetaLearnerMLP(nn.Module):
    def __init__(self, input_dim=9, hidden_dim=16, output_dim=3):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

print('MLP models defined')

MLP models defined


In [6]:
# Physical Constraint Normalization
def enforce_mass_balance(predictions):
    """
    Normalize predictions where sum > 1 to enforce mass balance constraint.
    predictions shape: (N, 3) for [Product 2, Product 3, SM]
    """
    predictions = predictions.copy()
    total = predictions.sum(axis=1, keepdims=True)
    mask = (total > 1).flatten()
    if mask.any():
        predictions[mask] = predictions[mask] / total[mask]
    return predictions

# Test the function
test_preds = np.array([[0.3, 0.4, 0.5], [0.2, 0.3, 0.4], [0.1, 0.1, 0.1]])
print('Before normalization:')
print(test_preds)
print(f'Sums: {test_preds.sum(axis=1)}')

normalized = enforce_mass_balance(test_preds)
print('\nAfter normalization:')
print(normalized)
print(f'Sums: {normalized.sum(axis=1)}')

Before normalization:
[[0.3 0.4 0.5]
 [0.2 0.3 0.4]
 [0.1 0.1 0.1]]
Sums: [1.2 0.9 0.3]

After normalization:
[[0.25       0.33333333 0.41666667]
 [0.2        0.3        0.4       ]
 [0.1        0.1        0.1       ]]
Sums: [1.  0.9 0.3]


In [7]:
# Stacking Ensemble with Physical Constraints
class StackingEnsembleWithConstraints:
    """Ensemble with MLP meta-learner and physical constraint enforcement."""
    def __init__(self, data='single', enforce_constraints=True):
        self.data = data
        self.enforce_constraints = enforce_constraints
        self.mixed = (data == 'full')
        
        self.full_featurizer = FullFeaturizer(mixed=self.mixed)
        self.simple_featurizer = SimpleFeaturizer(mixed=self.mixed)
        
        self.scaler_full = StandardScaler()
        self.scaler_simple = StandardScaler()
        self.gp_models = []
        self.mlp_model = None
        self.lgbm_models = []
        
        # Meta-learner weights (learned from OOF predictions)
        self.meta_weights = np.array([0.15, 0.55, 0.30])  # Default: GP, MLP, LGBM
        
    def train_model(self, X, Y):
        X_full = self.full_featurizer.featurize(X)
        X_simple = self.simple_featurizer.featurize(X)
        Y_np = Y.values
        
        X_full_scaled = self.scaler_full.fit_transform(X_full)
        X_simple_scaled = self.scaler_simple.fit_transform(X_simple)
        
        # Train GP for each target
        for i in range(3):
            kernel = ConstantKernel(1.0) * Matern(length_scale=1.0, nu=2.5) + WhiteKernel(noise_level=0.1)
            gp = GaussianProcessRegressor(
                kernel=kernel, alpha=0.01, n_restarts_optimizer=3, random_state=42
            )
            gp.fit(X_simple_scaled, Y_np[:, i])
            self.gp_models.append(gp)
        
        # Train MLP
        self.mlp_model = MLPModel(
            input_dim=self.full_featurizer.feats_dim,
            hidden_dim=64,
            output_dim=3,
            dropout=0.2
        ).to(device)
        
        X_tensor = torch.tensor(X_full_scaled, dtype=torch.double).to(device)
        Y_tensor = torch.tensor(Y_np, dtype=torch.double).to(device)
        
        optimizer = torch.optim.Adam(self.mlp_model.parameters(), lr=0.001, weight_decay=1e-4)
        criterion = nn.MSELoss()
        
        self.mlp_model.train()
        for epoch in range(200):
            optimizer.zero_grad()
            pred = self.mlp_model(X_tensor)
            loss = criterion(pred, Y_tensor)
            loss.backward()
            optimizer.step()
        
        # Train LGBM for each target
        for i in range(3):
            lgbm = lgb.LGBMRegressor(
                n_estimators=100, learning_rate=0.05, max_depth=5,
                num_leaves=31, min_child_samples=5, random_state=42, verbose=-1
            )
            lgbm.fit(X_full_scaled, Y_np[:, i])
            self.lgbm_models.append(lgbm)
        
        return self
    
    def predict(self, X):
        X_full = self.full_featurizer.featurize(X)
        X_simple = self.simple_featurizer.featurize(X)
        
        X_full_scaled = self.scaler_full.transform(X_full)
        X_simple_scaled = self.scaler_simple.transform(X_simple)
        
        # GP predictions
        gp_preds = np.column_stack([self.gp_models[i].predict(X_simple_scaled) for i in range(3)])
        
        # MLP predictions
        self.mlp_model.eval()
        X_tensor = torch.tensor(X_full_scaled, dtype=torch.double).to(device)
        with torch.no_grad():
            mlp_preds = self.mlp_model(X_tensor).cpu().numpy()
        
        # LGBM predictions
        lgbm_preds = np.column_stack([self.lgbm_models[i].predict(X_full_scaled) for i in range(3)])
        
        # Weighted ensemble (using default weights)
        predictions = 0.15 * gp_preds + 0.55 * mlp_preds + 0.30 * lgbm_preds
        
        # TTA for mixtures
        if self.mixed:
            X_full_flip = self.full_featurizer.featurize(X, flip=True)
            X_simple_flip = self.simple_featurizer.featurize(X, flip=True)
            
            X_full_scaled_flip = self.scaler_full.transform(X_full_flip)
            X_simple_scaled_flip = self.scaler_simple.transform(X_simple_flip)
            
            gp_preds_flip = np.column_stack([self.gp_models[i].predict(X_simple_scaled_flip) for i in range(3)])
            X_tensor_flip = torch.tensor(X_full_scaled_flip, dtype=torch.double).to(device)
            with torch.no_grad():
                mlp_preds_flip = self.mlp_model(X_tensor_flip).cpu().numpy()
            lgbm_preds_flip = np.column_stack([self.lgbm_models[i].predict(X_full_scaled_flip) for i in range(3)])
            
            predictions_flip = 0.15 * gp_preds_flip + 0.55 * mlp_preds_flip + 0.30 * lgbm_preds_flip
            predictions = (predictions + predictions_flip) / 2
        
        # Clip to [0, 1]
        predictions = np.clip(predictions, 0, 1)
        
        # Apply physical constraint normalization
        if self.enforce_constraints:
            predictions = enforce_mass_balance(predictions)
        
        return torch.tensor(predictions)

print('Stacking Ensemble with Constraints defined')

Stacking Ensemble with Constraints defined


In [8]:
# First, let's test the impact of physical constraint normalization alone
# by comparing CV with and without constraints

print('=== Testing Physical Constraint Normalization Impact ===')

X_single, Y_single = load_data("single_solvent")

# Run CV WITHOUT constraints
print('\n--- Without Physical Constraints ---')
split_generator = generate_leave_one_out_splits(X_single, Y_single)
all_preds_no_constraint = []
all_actuals = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=24):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = StackingEnsembleWithConstraints(data='single', enforce_constraints=False)
    model.train_model(train_X, train_Y)
    predictions = model.predict(test_X)
    
    all_preds_no_constraint.append(predictions.numpy())
    all_actuals.append(test_Y.values)

preds_no_constraint = np.vstack(all_preds_no_constraint)
actuals = np.vstack(all_actuals)
mse_no_constraint = np.mean((preds_no_constraint - actuals) ** 2)
print(f'MSE without constraints: {mse_no_constraint:.6f}')

# Check how many rows violate constraint
violations_no_constraint = (preds_no_constraint.sum(axis=1) > 1).sum()
print(f'Rows violating sum > 1: {violations_no_constraint} ({100*violations_no_constraint/len(preds_no_constraint):.1f}%)')

=== Testing Physical Constraint Normalization Impact ===

--- Without Physical Constraints ---


  0%|          | 0/24 [00:00<?, ?it/s]

  4%|▍         | 1/24 [00:33<12:46, 33.33s/it]

  8%|▊         | 2/24 [01:00<10:54, 29.75s/it]

 12%|█▎        | 3/24 [01:22<09:09, 26.15s/it]

...

 92%|█████████▏| 22/24 [10:05<00:55, 27.68s/it]

 96%|█████████▌| 23/24 [10:34<00:28, 28.07s/it]

100%|██████████| 24/24 [11:01<00:00, 27.69s/it]

100%|██████████| 24/24 [11:01<00:00, 27.57s/it]

MSE without constraints: 0.009663
Rows violating sum > 1: 26 (4.0%)


In [9]:
# Now apply constraints and see the impact
print('\n--- With Physical Constraints ---')
preds_with_constraint = enforce_mass_balance(preds_no_constraint)
mse_with_constraint = np.mean((preds_with_constraint - actuals) ** 2)
print(f'MSE with constraints: {mse_with_constraint:.6f}')

# Check violations after constraint
violations_with_constraint = (preds_with_constraint.sum(axis=1) > 1).sum()
print(f'Rows violating sum > 1: {violations_with_constraint}')

# Compare
if mse_with_constraint < mse_no_constraint:
    improvement = (mse_no_constraint - mse_with_constraint) / mse_no_constraint * 100
    print(f'\n✓ Physical constraints IMPROVE MSE by {improvement:.2f}%')
else:
    degradation = (mse_with_constraint - mse_no_constraint) / mse_no_constraint * 100
    print(f'\n✗ Physical constraints WORSEN MSE by {degradation:.2f}%')


--- With Physical Constraints ---
MSE with constraints: 0.009658
Rows violating sum > 1: 0

✓ Physical constraints IMPROVE MSE by 0.05%


In [10]:
# Run full CV with constraints on both datasets
print('\n=== Full CV with Physical Constraints ===')

# Single solvent
print('\n--- Single Solvent CV ---')
X_single, Y_single = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X_single, Y_single)
all_predictions_single = []
all_actuals_single = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=24):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = StackingEnsembleWithConstraints(data='single', enforce_constraints=True)
    model.train_model(train_X, train_Y)
    predictions = model.predict(test_X)
    
    all_predictions_single.append(predictions.numpy())
    all_actuals_single.append(test_Y.values)

preds_single = np.vstack(all_predictions_single)
actuals_single = np.vstack(all_actuals_single)
mse_single = np.mean((preds_single - actuals_single) ** 2)
print(f'Single Solvent MSE: {mse_single:.6f} (n={len(preds_single)})')


=== Full CV with Physical Constraints ===

--- Single Solvent CV ---


  0%|          | 0/24 [00:00<?, ?it/s]

  4%|▍         | 1/24 [00:26<10:04, 26.28s/it]

  8%|▊         | 2/24 [00:52<09:33, 26.05s/it]

 12%|█▎        | 3/24 [01:13<08:24, 24.02s/it]

...

 92%|█████████▏| 22/24 [10:02<00:55, 27.68s/it]

 96%|█████████▌| 23/24 [10:31<00:27, 27.84s/it]

100%|██████████| 24/24 [10:58<00:00, 27.71s/it]

100%|██████████| 24/24 [10:58<00:00, 27.44s/it]

Single Solvent MSE: 0.009416 (n=656)


In [ ]:
# Full data CV
print('\n--- Full Data CV ---')
X_full, Y_full = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X_full, Y_full)
all_predictions_full = []
all_actuals_full = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=13):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = StackingEnsembleWithConstraints(data='full', enforce_constraints=True)
    model.train_model(train_X, train_Y)
    predictions = model.predict(test_X)
    
    all_predictions_full.append(predictions.numpy())
    all_actuals_full.append(test_Y.values)

preds_full = np.vstack(all_predictions_full)
actuals_full = np.vstack(all_actuals_full)
mse_full = np.mean((preds_full - actuals_full) ** 2)
print(f'Full Data MSE: {mse_full:.6f} (n={len(preds_full)})')

In [ ]:
# Calculate overall MSE
n_single = len(preds_single)
n_full = len(preds_full)
overall_mse = (mse_single * n_single + mse_full * n_full) / (n_single + n_full)

print(f'\n=== CV SCORE SUMMARY (Physical Constraints) ===')
print(f'Single Solvent MSE: {mse_single:.6f} (n={n_single})')
print(f'Full Data MSE: {mse_full:.6f} (n={n_full})')
print(f'Overall MSE: {overall_mse:.6f}')
print(f'\nBest CV (exp_035): 0.008194')

if overall_mse < 0.008194:
    improvement = (0.008194 - overall_mse) / 0.008194 * 100
    print(f'\n✓ IMPROVEMENT: {improvement:.2f}% better than best CV!')
else:
    degradation = (overall_mse - 0.008194) / 0.008194 * 100
    print(f'\n✗ WORSE: {degradation:.2f}% worse than best CV')

# Check constraint violations in final predictions
all_preds = np.vstack([preds_single, preds_full])
violations = (all_preds.sum(axis=1) > 1.001).sum()  # Small tolerance for floating point
print(f'\nConstraint violations in final predictions: {violations}')

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

import tqdm

X, Y = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = StackingEnsembleWithConstraints(data='single', enforce_constraints=True)  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_single_solvent = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = StackingEnsembleWithConstraints(data='full', enforce_constraints=True)  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_full_data = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
# Final verification
print(f'\n=== FINAL CV SCORE ===')
print(f'Single Solvent MSE: {mse_single:.6f} (n={n_single})')
print(f'Full Data MSE: {mse_full:.6f} (n={n_full})')
print(f'Overall MSE: {overall_mse:.6f}')
print(f'\nBest CV (exp_035): 0.008194')